In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from parseit.data import load_pickle
import pandas as pd

pd.options.display.min_rows = 100
#pd.options.display.max_rows = set larger than dataset
pd.options.display.max_columns = 20
%matplotlib inline
mpl.rcParams["figure.figsize"] = 15,8

#pickle_df = load_pickle("comments.p")
#pickle_df = load_pickle("okt-5-10728c.p")
#pickle_df = load_pickle("okt-5-43896c.p")
pickle_df = load_pickle("okt-11-4788c.p")

In [2]:
# Make a copy to work from original
df = pickle_df.copy()

# remove noise
df = df[df["body"] != "[removed]"]
df = df.reset_index()

# Feature extraction
from modules.bag_of_words import BagOfWords
from modules.bigram_of_words import BigramOfWords
from modules.tfidf import TfIdf

df = BigramOfWords.score(df)
#df = BagOfWords.score(df)
#df = TfIdf.score(df)
display(df)

# benfords law
# https://www.youtube.com/watch?v=etx0k1nLn78

,index,body,subreddit,submission,label,! !,! &,! ',! 'm,! (,...,🎵 peking,🎵we went,👌 lgbt,😂 know,😄🇿🇦also supersized,😢 'm,😻 4,🙄 ),🙄 /s,🤣 born
0,0,*What if Jod was one of us?*,askreddit,"What if God came down one day and said ""It's p...",759,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,A large chunk of my taking the lord's name in ...,askreddit,"What if God came down one day and said ""It's p...",41875,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,Do you want a Holy War? Because that's how you...,askreddit,"What if God came down one day and said ""It's p...",26321,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,"He would also say, “my name isn’t God” then he...",askreddit,"What if God came down one day and said ""It's p...",6587,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,"We’d be like, “No way!” and he’d be like, “Yah...",askreddit,"What if God came down one day and said ""It's p...",20310,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,I would be converted\n\n\nEdit: thanks for the...,askreddit,"What if God came down one day and said ""It's p...",6656,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,What if Satan came down one day and said ''It'...,askreddit,"What if God came down one day and said ""It's p...",5339,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,7,Then all of Scandinavia will pronounce it like...,askreddit,"What if God came down one day and said ""It's p...",15878,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,8,You'd have different religious factions arguin...,askreddit,"What if God came down one day and said ""It's p...",15221,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,Does that mean Jesus would be pronounced Geezus?,askreddit,"What if God came down one day and said ""It's p...",23071,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import MinMaxScaler, KBinsDiscretizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import metrics
import numpy as np

# Drop indexs more from lower score and less from higher score
# https://stackoverflow.com/questions/28556942/pandas-remove-rows-at-random-without-shuffling-dataset/28557333#28557333
# np.random.seed(10)


y = df[["label"]]
X = df.drop(["body", "submission", "subreddit", "label"], axis=1)


# Bin all of y first
# uniform = All bins in each feature have identical widths.
# quantile = All bins in each feature have the same number of points.
# kmeans = Values in each bin have the same nearest center of a 1D k-means cluster.
total_bins = 10
for bins in range(2, total_bins+1, 2):
    # kmeans seems right: https://scikit-learn.org/stable/auto_examples/preprocessing/plot_discretization_strategies.html
    est = KBinsDiscretizer(n_bins=bins, encode='ordinal', strategy="kmeans") #
    est.fit(y)
    yt = pd.DataFrame(data=est.transform(y), columns=["y"])

    # Drop some data
    to_drop = yt[yt.y == 0][:500]
    yt_filtered = yt.drop(index=to_drop.index)
    X_filtered = X.drop(index=to_drop.index)


    # Bad idea to do random split with so skewed data?
    #data = scaler.transform(data)
    #X_train, X_validate, y_train, y_validate = train_test_split(X, yt, test_size=0.7) 

    # https://stats.stackexchange.com/questions/131255/class-imbalance-in-supervised-machine-learning
    # https://datascience.stackexchange.com/questions/32818/train-test-split-of-unbalanced-dataset-classification/32820#32820
    # https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html
    skf = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)
    # skf.get_n_splits(X, yt)
    balanced_accuracies = []
    accuracies = []
    predictions = np.array([])
    true_values = np.array([])

    for train_index, test_index in skf.split(X_filtered, yt_filtered):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_validate = X_filtered.iloc[train_index], X_filtered.iloc[test_index]
        y_train, y_true = yt_filtered.iloc[train_index], yt_filtered.iloc[test_index]

        #scaler = MinMaxScaler(min=0, max=1)
        scaler = MinMaxScaler()
        scaler.fit(X_train, y_train) # Don't cheat: fit only training data. y is ignored, but needs same shape[rows]

        model = MultinomialNB()

        # Train the model using the training sets - scale only features (?)
        model.fit(scaler.transform(X_train), y_train)

        # Predic the model
        y_pred = model.predict(scaler.transform(X_validate))

        accuracy = metrics.accuracy_score(y_true, y_pred, normalize=True)
        # TODO: balanced - https://stackoverflow.com/questions/55838262/why-does-cross-validation-give-consistently-higher-scores-than-normal-fitting-an
        balanced_accuracy = metrics.balanced_accuracy_score(y_true, y_pred, adjusted=False)
        # TODO: https://stats.stackexchange.com/questions/99667/naive-bayes-with-unbalanced-classes

        accuracies.append(accuracy)
        balanced_accuracies.append(balanced_accuracy)
        predictions = np.append(predictions, y_pred)
        true_values = np.append(true_values, y_true)

        # TODO: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.balanced_accuracy_score.html

    results = pd.DataFrame(data={"pred": predictions, "true": true_values})
    counts = pd.DataFrame(data={
        f"true-{bins}": results["true"].value_counts(dropna=False),
        f"pred-{bins}": results["pred"].value_counts(normalize=False, dropna=False),
        f"percent-{bins}": results["pred"].value_counts(normalize=True, dropna=False)
    })
    counts = pd.concat([pd.DataFrame(data=[{
        f"true-{bins}": len(df) - results["true"].value_counts(dropna=False).sum(),
        f"pred-{bins}": len(df) - results["pred"].value_counts(dropna=False).sum(),
        f"percent-{bins}": (len(df) - results["pred"].value_counts(dropna=False).sum()) / (len(df) - results["true"].value_counts(dropna=False).sum())
    }]), counts], ignore_index=True)

    print(f"\n\nMean: {np.mean(accuracies)} \n Mean balanced: {np.mean(balanced_accuracies)}")
    display(counts)




Mean: 0.6342072409488139 
 Mean balanced: 0.5864739656230407


,true-2,pred-2,percent-2
0,500,500,1.000000
1,3812,2527,0.630961
2,193,1478,0.369039




Mean: 0.6009987515605494 
 Mean balanced: 0.2835099848602609


,true-4,pred-4,percent-4
0,500,500,1.000000
1,3641,2476,0.618227
2,244,1170,0.292135
3,91,305,0.076155
4,29,54,0.013483




Mean: 0.599250936329588 
 Mean balanced: 0.2007950321944675


,true-6,pred-6,percent-6
0,500,500.0,1.000000
1,3617,2493.0,0.622472
2,229,1144.0,0.285643
3,103,258.0,0.064419
4,37,104.0,0.025968
5,17,6.0,0.001498
6,2,NaN,NaN




Mean: 0.5967540574282147 
 Mean balanced: 0.16358771995517715


,true-8,pred-8,percent-8
0,500,500.0,1.000000
1,3538,2531.0,0.631960
2,216,883.0,0.220474
3,130,439.0,0.109613
4,67,92.0,0.022971
5,29,53.0,0.013233
6,15,5.0,0.001248
7,8,2.0,0.000499
8,2,NaN,NaN




Mean: 0.5920099875156054 
 Mean balanced: 0.11812353935314336


,true-10,pred-10,percent-10
0,500,500.0,1.000000
1,3451,2568.0,0.641199
2,202,731.0,0.182522
3,141,364.0,0.090886
4,101,236.0,0.058926
5,56,65.0,0.016230
6,26,33.0,0.008240
7,13,8.0,0.001998
8,8,NaN,NaN
9,5,NaN,NaN
